In [3]:
import rasterio
import rasterio.features
import rasterio.warp
import geopyspark as gps
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from geonotebook.wrappers import TMSRasterData
from osgeo import osr

import os
import math
import boto3

%matplotlib inline

In [4]:
conf = gps.geopyspark_conf("yarn-client", "SRTM Ingest") \
          .set("spark.dynamicAllocation.enabled", "false") \
          .set("spark.executor.instances", "50") \
          .set("spark.executor.memory", "9472M") \
          .set("spark.executor.cores", "4") \
          .set("spark.ui.enabled","true") \
          .set("spark.hadoop.yarn.timeline-service.enabled", False)

In [5]:
sc = SparkContext(conf=conf)

In [6]:
s3 = boto3.client('s3')
def get_raster_s3_objects(bucket, prefix, extension="hgt"):
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket, Prefix=prefix)
    results = []
    for page in page_iterator:
        for item in page['Contents']:
            if item['Key'].endswith(extension):
                results.append(item)
    return results


In [7]:
object_names = get_raster_s3_objects("mrgeo-source", "srtm-v3-30")

In [8]:
file_names = list(map(lambda d: d['Key'][len('srtm-v3-30/'):], object_names))
print(len(file_names))
print(file_names[0:10])

14265
['N00E006.hgt', 'N00E009.hgt', 'N00E010.hgt', 'N00E011.hgt', 'N00E012.hgt', 'N00E013.hgt', 'N00E014.hgt', 'N00E015.hgt', 'N00E016.hgt', 'N00E017.hgt']


In [9]:
def get_metadata(uri):
    import rasterio
    from osgeo import osr
    import os
    
    if "GDAL_DATA" not in os.environ:
        os.environ["GDAL_DATA"]="/usr/local/lib64/python3.4/site-packages/fiona/gdal_data"
    
    try:
        with rasterio.open(uri) as dataset:
            bounds = dataset.bounds
            height = dataset.height
            width = dataset.width
            crs = dataset.get_crs()
            srs = osr.SpatialReference()
            srs.ImportFromWkt(crs.wkt)
            proj4 = srs.ExportToProj4()
            # ws = [w for (ij, w) in dataset.block_windows()]
            tile_cols = math.floor((width - 1) / 512) * 512 # XXX
            tile_rows = math.floor((height - 1) / 512) * 512 # XXX
            ws = [((x, x + 512), (y, y + 512)) for x in range(0, tile_cols, 512) for y in range(0, tile_rows, 512)]
    except:
            ws = []
            
    def windows(uri, ws):
        for w in ws:
            ((row_start, row_stop), (col_start, col_stop)) = w

            left  = bounds.left + (bounds.right - bounds.left)*(float(col_start)/width)
            right = bounds.left + (bounds.right - bounds.left)*(float(col_stop)/ width)
            bottom = bounds.top + (bounds.bottom - bounds.top)*(float(row_stop)/height)
            top = bounds.top + (bounds.bottom - bounds.top)*(float(row_start)/height)
            extent = gps.Extent(left,bottom,right,top)
                
            new_line = {}
            new_line['uri'] = uri
            new_line['window'] = w
            new_line['projected_extent'] = gps.ProjectedExtent(extent=extent, proj4=proj4)
            yield new_line
    
    return [i for i in windows(uri, ws)]


In [10]:
def get_data(line):
    import rasterio
    
    new_line = line.copy()

    with rasterio.open(line['uri']) as dataset:
        new_line['data'] = dataset.read(1, window=line['window'])
        new_line.pop('window')
        new_line.pop('uri')
    
    return new_line

In [11]:
def filename_to_data(filename):
    import os
    
    full_filename = "/vsicurl/https://s3.amazonaws.com/mrgeo-source/srtm-v3-30/{}".format(filename)
    data = [get_data(line) for line in get_metadata(full_filename)]
    return data

In [12]:
rdd0 = sc.parallelize(file_names)
rdd1 = rdd0.flatMap(filename_to_data)
print(rdd1.count())

698985


In [13]:
rdd2 = rdd1.groupBy(lambda line: line['projected_extent']) # XXX

In [14]:
def make_tiles(line):
    projected_extent = line[0]
    array = np.array([l['data'] for l in line[1]])
    tile = gps.Tile.from_numpy_array(array, no_data_value=0)
    return (projected_extent, tile)

def interesting_tile(line):
    [pe, tile] = line
    return (np.sum(tile[0][0]) != 0)

def square_tile(line):
    [pe, tile] = line
    return tile[0][0].shape == (512,512)

In [15]:
# rdd3 = rdd2.repartition(50 * 1024).map(make_tiles).filter(square_tile)
rdd3 = rdd2.repartition(50 * 1024).map(make_tiles)

In [16]:
raster_layer = gps.RasterLayer.from_numpy_rdd(gps.LayerType.SPATIAL, rdd3)

In [17]:
tiled_raster_layer = raster_layer.tile_to_layout(layout = gps.GlobalLayout(), target_crs=3857)

In [18]:
pyramid = tiled_raster_layer.pyramid()

In [19]:
for layer in pyramid.levels.values():
    gps.write("s3://geotrellis-test/dg-srtm/", "srtm-geopyspark", layer)

Py4JJavaError: An error occurred while calling o143.writeSpatial.
: geotrellis.spark.io.package$LayerWriteError: Failed to write Layer(name = "srtm-geopyspark", zoom = 12)
	at geotrellis.spark.io.s3.S3LayerWriter._write(S3LayerWriter.scala:123)
	at geotrellis.spark.io.s3.S3LayerWriter._write(S3LayerWriter.scala:49)
	at geotrellis.spark.io.LayerWriter$class.write(LayerWriter.scala:153)
	at geotrellis.spark.io.s3.S3LayerWriter.write(S3LayerWriter.scala:49)
	at geopyspark.geotrellis.io.LayerWriterWrapper.writeSpatial(LayerWriterWrapper.scala:91)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Ga